# Linear Regression for feature ranking

In [58]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


import os
# change to your local path to project below
project_path = '/home/jorge/Insync/jorgitoje@gmail.com/OneDrive/Documentos/JORGE/EDUCATION/MASTER_DATASCIENCE/Semester1/AdvancedStatistics/GroupProject/KDS_Statistics_GroupProject'
# TO DO:
# use preprocessed dataset from Janos and Mateusz, for now I did it myself

In [59]:
dataset_path = os.path.join(project_path, 'datasets/USA_houseprices_2014/data_processed.csv')
df = pd.read_csv(dataset_path)
df['yrs_lastmod'] = 2022 - df['last_modification']
df.head()


,Unnamed: 0,price,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,street,city,statezip,country,last_modification,have_basement,yrs_lastmod
0,0,313000.0,2014-05-02 00:00:00,3.0,1.50,1340,7912,1.5,0,0,...,0,1955,2005,1522,36,62,0,2005,1,17
1,1,2384000.0,2014-05-02 00:00:00,5.0,2.50,3650,9050,2.0,0,4,...,280,1921,0,3899,35,58,0,1921,1,101
2,2,342000.0,2014-05-02 00:00:00,3.0,2.00,1930,11947,1.0,0,0,...,0,1966,0,2291,18,26,0,1966,1,56
3,3,420000.0,2014-05-02 00:00:00,3.0,2.25,2000,8030,1.0,0,0,...,1000,1963,0,4263,3,7,0,1963,1,59
4,4,550000.0,2014-05-02 00:00:00,4.0,2.50,1940,10500,1.0,0,0,...,800,1976,1992,4352,31,31,0,1992,1,30


In [60]:
# Define dictionary to store our rankings
ranks = {}
# Create our function which stores the feature rankings to the ranks dictionary
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))

In [61]:
df_regression = df.drop(['Unnamed: 0', 'date', 'last_modification', 'yr_built', 'yr_renovated' ],axis=1)
df_regression.head()
y = df_regression['price']
X = df_regression.drop(['price'], axis = 1)
min_max_scaler = MinMaxScaler().fit(X)
X_norm = min_max_scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size = .2, random_state = 1)
colnames = X.columns



In [62]:
lr = LinearRegression()
lr.fit(X_train,y_train)
ranks["LinReg"] = ranking(np.abs(lr.coef_), colnames)

# Using Ridge 
ridge = Ridge(alpha = 7)
ridge.fit(X_train,y_train)
ranks['Ridge'] = ranking(np.abs(ridge.coef_), colnames)

# Using Lasso
lasso = Lasso(alpha=.05)
lasso.fit(X_train,y_train)
ranks["Lasso"] = ranking(np.abs(lasso.coef_), colnames)

/home/jorge/Insync/jorgitoje@gmail.com/OneDrive/Documentos/JORGE/EDUCATION/MASTER_DATASCIENCE/Semester1/AdvancedStatistics/GroupProject/KDS_Statistics_GroupProject/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.450e+14, tolerance: 1.349e+11
  model = cd_fast.enet_coordinate_descent(


In [63]:
rf = RandomForestRegressor(n_jobs=-1, n_estimators=50, verbose=3)
rf.fit(X, y)
ranks["RF"] = ranking(rf.feature_importances_, colnames);

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


building tree 1 of 50building tree 2 of 50
building tree 3 of 50
building tree 4 of 50

building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished


In [64]:
r = {}
for name in colnames:
    r[name] = round(np.mean([ranks[method][name] 
                             for method in ranks.keys()]), 2)
 
methods = sorted(ranks.keys())
ranks["Mean"] = r
methods.append("Mean")
 
print("\t\t%s" % "\t".join(methods))
for name in colnames:
    print("%s\t%s" % (name, "\t".join(map(str, 
                         [ranks[method][name] for method in methods]))))

		Lasso	LinReg	RF	Ridge	Mean
bedrooms	0.11	0.0	0.04	0.12	0.07
bathrooms	0.06	0.0	0.22	0.35	0.16
sqft_living	1.0	1.0	0.99	0.86	0.96
sqft_lot	0.2	0.0	0.26	0.17	0.16
floors	0.0	0.0	0.03	0.03	0.02
waterfront	0.17	0.0	0.02	0.44	0.16
view	0.05	0.0	0.06	0.22	0.08
condition	0.06	0.0	0.02	0.17	0.06
sqft_above	0.01	0.69	0.18	1.0	0.47
sqft_basement	0.03	0.37	0.08	0.48	0.24
street	0.0	0.0	1.0	0.01	0.25
city	0.04	0.0	0.13	0.14	0.08
statezip	0.01	0.0	0.27	0.05	0.08
country	0.0	0.0	0.0	0.0	0.0
have_basement	0.0	0.0	0.0	0.0	0.0
yrs_lastmod	0.0	0.0	0.2	0.02	0.06
